In [1]:
# Notebook para generar el dataset de parcelas

In [2]:
import os
import numpy as np
import pandas as pd

import geopandas as gpd
from fiona.drvsupport import supported_drivers
supported_drivers['KML'] = 'rw'


In [3]:
# Rutas
ROOT_PATH = 'C:/Users/pedro/ds-geo'
CONFIG_PATH = ROOT_PATH + '/config'
DATA_PATH = ROOT_PATH + '/data'
PARCELAS_PATH = ROOT_PATH + '/data/parcelas'

DATASET_PARCELAS = 'dataset_parcelas.pkl'

os.chdir(ROOT_PATH)

In [4]:
# Carga de todos los ficheros del catastro, en formato KML.

def LoadCatastroKML(s_file):
    file_polygons = gpd.read_file(PARCELAS_PATH + '/' + s_file + '.kml', driver='KML')
    df_parc = pd.DataFrame(columns=['CatReferencia', 'CatRecinto', 'Latitude', 'Longitude'])
    
    for index, row in file_polygons.iterrows():
        l_lat = []
        l_long = []
        if len(row['Name']) > 0:
            s_sub = row['Name']
        else:
            s_sub = 'z'
        
        for point in list(row['geometry'].exterior.coords): 
            l_lat.append(point[1])
            l_long.append(point[0])

        row_new = {'CatReferencia': s_file, 'CatRecinto': s_sub, 'Latitude': l_lat, 'Longitude': l_long}
        df_new = pd.DataFrame([row_new])

        df_parc = pd.concat([df_parc, df_new], axis=0, ignore_index=True)
    
    return df_parc


In [5]:
# Fución para cargar todos los ficheros de parcelas del Catastro (desde una carpeta)
def LoadCatastro():
    
    files_list = []
    for file in os.listdir(PARCELAS_PATH):
        if file.endswith(".kml"):
            files_list.append(file.replace('.kml', ''))

    df_parcelas = pd.DataFrame(columns=['CatReferencia', 'CatRecinto', 'Latitude', 'Longitude'])

    for file in files_list:
        df_new = LoadCatastroKML(file)
        df_parcelas = pd.concat([df_parcelas, df_new], axis=0, ignore_index=True)

    return df_parcelas


In [6]:
# Carga información de las parcelas

def LoadParcelasInfo():
    
    df_infoA = pd.read_csv(PARCELAS_PATH + '/parcelas.csv', sep=',', encoding='UTF-8', na_filter= False)
    df_infoC = pd.read_csv(CONFIG_PATH + '/cultivos.cfg', sep=',', encoding='UTF-8', na_filter= False)
    df_infoP = pd.read_csv(CONFIG_PATH + '/propietarios.cfg', sep=',', encoding='UTF-8', na_filter= False)
    
    df_info = pd.merge(df_infoA, df_infoC, on=['CodigoCultivo'], how='left')
    df_info = pd.merge(df_info, df_infoP, on=['CodigoPropietario'], how='left')
    
    df_info.fillna('', inplace=True)
    df_info.drop(['CodigoCultivo', 'CodigoPropietario'], axis=1, inplace=True)
    
    return df_info
    

In [7]:
# Función que guarda el dataset de las parcelas para ser visualizado en Google Maps

def SaveDataset(df):
    df.to_pickle(DATA_PATH + '/' + DATASET_PARCELAS)
 

In [8]:
# Función que fusiona la información del Catastro con la propia de la empresa 
# para generar el dataset que se visualizará postariormente

def GenerateDatasetParcelas():
    # Cargamos las parcelas del Catastro
    df_parcelas = LoadCatastro()
  
    # Cargamos la información complementaria de las parcelas
    df_infoParcelas = LoadParcelasInfo()   

    # Completamos la información de las parcelas
    df_compl = pd.merge(df_parcelas, df_infoParcelas, on=['CatReferencia', 'CatRecinto'], how='left')
    df_compl.fillna('', inplace=True)
    
    # Volcamos a disco el dataset
    SaveDataset(df_compl)


## Ejecución

In [9]:
# Generación del dataset de parcelas
GenerateDatasetParcelas()